# Idea
Do some training

In [1]:
SEED = 133

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [6]:
import sys
repo_base_path = Path("./../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

# from py_dataset import get_all_files_df
# from py_dataset import read_in_files
from py_dataset import feature_plotting
# from py_dataset import feature_selection
# from py_dataset import net_feature_engineering
# from py_dataset import entropy_feature_engineering
# from py_dataset.classes import DataSource
from py_dataset import sys_func



In [8]:
df = pd.read_csv('../data/all_df.csv')
df.set_index("timestamp", inplace=True)

In [9]:
df["label"].unique()

array(['1_normal', nan, '2_ransomware', '3_thetick', '4_bashlite',
       '5_httpbackdoor', '6_beurk', '7_backdoor', '8_bdvl', '9_xmrig'],
      dtype=object)

### Preprocessings
- remove Null values
- drop timestamp
- labelEncode
- normalize

In [10]:
print(df.shape)
old_df = df.copy()
df.dropna(axis=0, how='any', inplace=True)

print(df.shape)

(43248, 306)
(25770, 306)


In [11]:
df.reset_index(inplace=True, drop=True)

In [12]:
label_encoder = sys_func.create_label_encoder_for_labels()
df['label'] = label_encoder.transform(df['label'])

In [13]:
df = feature_plotting.test_transform_all_to_numeric_columns(df)

In [14]:
cols_to_exclude_from_scaling = ["label"]
vectors = feature_plotting.minmax_scale_features(df,remove_outliers_iqr_all_columns=False,
                                                        cols_to_exclude_from_scaling=cols_to_exclude_from_scaling)
# print(vectors.shape)
# print(" with dropped null: ", vectors.dropna(axis=0, how='any', inplace=False).shape)

In [15]:
df["label"].unique(), vectors["label"].unique()

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([0, 1, 2, 3, 4, 5, 6, 7, 8]))

### Setup

In [22]:
train.drop(columns=["label"])

,ext4:ext4_da_write_begin,jbd2:jbd2_drop_transaction,writeback:sb_clear_inode_writeback_FLS_data,jbd2:jbd2_handle_extend,ext4:ext4_truncate_exit,seconds_FLS_data,block:block_rq_issue,writeback:writeback_dirty_page_FLS_data,jbd2:jbd2_handle_start_FLS_data,block:block_bio_queue,...,sendto,unlink,sched_yield,recvfrom,fstatat64,close,getxattr,pause,inotify_add_watch,getuid32
21679,0.990926,0.333333,0.355932,0.5,0.666667,0.078208,0.096866,0.994343,0.880918,0.153203,...,0.029988,0.002978,0.0,0.037101,0.028835,0.531786,0.168505,0.126660,0.000000,0.511321
8582,0.000000,0.000000,0.084746,0.0,0.025641,0.057334,0.014245,0.001886,0.001076,0.030641,...,0.044504,0.009026,0.0,0.051305,0.049881,0.060895,0.053772,0.108403,0.000258,0.008252
2720,0.031915,0.333333,0.389831,0.0,0.538462,0.010907,0.079772,0.010057,0.044476,0.103064,...,0.176925,0.549808,0.0,0.170864,0.164042,0.657241,0.217962,0.000000,0.001134,0.186112
20089,0.013141,0.166667,0.135593,0.0,0.307692,0.061255,0.042735,0.003143,0.284433,0.097493,...,0.056014,0.008752,0.0,0.072696,0.053176,0.821527,0.054700,0.000000,0.000221,0.850201
7506,0.000626,0.166667,0.033898,0.0,0.076923,0.040112,0.011396,0.001886,0.003407,0.025070,...,0.048540,0.012316,0.0,0.051179,0.061283,0.068058,0.101015,0.104831,0.000249,0.007668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24661,0.000626,0.000000,0.033898,0.0,0.051282,0.044854,0.005698,0.000629,0.002511,0.005571,...,0.020625,0.000000,0.0,0.016848,0.014041,0.036296,0.000000,0.000000,0.000244,0.011966
4660,0.031602,0.166667,0.338983,0.5,0.512821,0.012911,0.065527,0.007857,0.042683,0.077994,...,0.237105,0.523618,0.0,0.237465,0.264057,0.744164,0.691916,0.431549,0.000513,0.186410
18284,0.000000,0.166667,0.000000,0.0,0.000000,0.038656,0.005698,0.001886,0.000000,0.022284,...,0.042365,0.005867,0.0,0.051459,0.042613,0.063488,0.118390,0.100844,0.000240,0.006020
24246,0.000000,0.333333,0.000000,0.0,0.000000,0.045413,0.005698,0.000314,0.000000,0.022284,...,0.005998,0.000000,0.0,0.007086,0.010232,0.023498,0.000000,0.000000,0.000120,0.008366


In [24]:
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# tensorboard = pl_loggers.TensorBoardLogger('./')

# split into train and val
train, val = train_test_split(vectors, test_size=0.2, random_state=SEED)
X_train = train.drop(columns=["label"])
y_train = train["label"]

estimator = LogisticRegression(max_iter=10000, multi_class='multinomial', verbose=1)
selector = RFE(estimator, n_features_to_select=30, step=1)
selector = selector.fit(X_train, y_train)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         2754     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.19722D+00    |proj g|=  9.57358D-02


 This problem is unconstrained.



At iterate   50    f=  4.06609D-01    |proj g|=  5.03700D-03

At iterate  100    f=  3.23653D-01    |proj g|=  5.90213D-03

At iterate  150    f=  2.82877D-01    |proj g|=  2.58575D-03

At iterate  200    f=  2.73689D-01    |proj g|=  5.64691D-04

At iterate  250    f=  2.71392D-01    |proj g|=  1.38858D-03

At iterate  300    f=  2.70592D-01    |proj g|=  1.80258D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 2754    348    375      1     0     0   8.675D-05   2.703D-01
  F =  0.27033138625286529     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machin

 This problem is unconstrained.



At iterate   50    f=  4.07044D-01    |proj g|=  4.43612D-03

At iterate  100    f=  3.22484D-01    |proj g|=  2.57791D-03

At iterate  150    f=  2.89920D-01    |proj g|=  2.57330D-03

At iterate  200    f=  2.73944D-01    |proj g|=  6.49490D-04

At iterate  250    f=  2.70949D-01    |proj g|=  3.65345D-04

At iterate  300    f=  2.70540D-01    |proj g|=  2.67083D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 2745    304    331      1     0     0   8.907D-05   2.705D-01
  F =  0.27052857563158617     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machin

...


At iterate   50    f=  3.95401D-01    |proj g|=  1.95543D-03

At iterate  100    f=  3.80210D-01    |proj g|=  5.92136D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  297    124    134      1     0     0   8.606D-05   3.799D-01
  F =  0.37988252273042039     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          288     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.19722D+00    |proj g|=  9.16170D-02


 This problem is unconstrained.



At iterate   50    f=  3.99532D-01    |proj g|=  5.01576D-03

At iterate  100    f=  3.81963D-01    |proj g|=  2.78983D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  288    123    135      1     0     0   8.825D-05   3.815D-01
  F =  0.38152185577294756     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          279     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.19722D+00    |proj g|=  9.16170D-02


 This problem is unconstrained.



At iterate   50    f=  4.00189D-01    |proj g|=  4.88198D-03

At iterate  100    f=  3.83285D-01    |proj g|=  4.78927D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  279    146    160      1     0     0   9.981D-05   3.828D-01
  F =  0.38280030842419338     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [25]:
selected_features = selector.support_
selected_indices = np.where(selected_features)[0]
print(f"Selected feature indices (top 30): {selected_indices}")
print(f"Number of selected features: {len(selected_indices)}")

Selected feature indices (top 30): [ 36  55  67  73 105 108 112 113 114 129 158 163 169 174 189 191 200 206
 213 222 242 248 253 257 264 279 282 288 290 304]
Number of selected features: 30


In [41]:
np.all(selected_indices == np.where(selector.ranking_==1))

True

In [67]:
selected_indices.tofile("selected_indices.txt")

In [77]:
subset = vectors.iloc[:,selected_indices].copy()
subset["label"] = vectors["label"]

In [80]:
subset.iloc[:,0].max()

1.0

In [81]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomeDataset(Dataset):
    def __init__(self, some_df:pd.DataFrame):
        
        self.x = torch.tensor(some_df.drop(columns=["label"], inplace=False).values, dtype=torch.float32)
        self.labels = torch.tensor(some_df["label"].values, dtype=torch.uint8)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # print(self.x[idx], self.labels[idx])
        return self.x[idx], self.labels[idx]

In [82]:
sys.path.append(str(repo_base_path  / "training"))

In [83]:
train_dataset.labels.shape

torch.Size([20616])

In [84]:
# tensorboard = pl_loggers.TensorBoardLogger('./')
from mlp import MalwaresModelMLP

from torch.utils.data import DataLoader
import lightning
from sklearn.model_selection import train_test_split
from lightning import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint


# split into train and val
train, val = train_test_split(subset, test_size=0.2, random_state=SEED)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=10, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True) #logger=None, 


model = MalwaresModelMLP(input_size = subset.shape[1]-1, output_size = len(label_encoder.classes_))

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [85]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 930   
6 | l2            | Linear                    | 930   
7 | l3            | Linear                    | 279   
------------------------------------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[  0,   0,   0,   0,   0,   0, 104,   0,   0],
        [  0,   0,   0,   0,   0,   0, 118,   0,   0],
        [  0,   0,   0,   0,   0,   0, 125,   0,   0],
        [  0,   0,   0,   0,   0,   0, 114,   0,   0],
        [  0,   0,   0,   0,   0,   0, 104,   0,   0],
        [  0,   0,   0,   0,   0,   0, 115,   0,   0],
        [  0,   0,   0,   0,   0,   0, 116,   0,   0],
        [  0,   0,   0,   0,  80,   0,  10,   0,   0],
        [  0,   0,   0,   0,   0,   0, 114,   0,   0]])


/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (42) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Epoch end: Validation, epoch number: 107
ValidationEpoch/CM
 tensor([[266511,      1,    256,   3965,   1002,      0,    913,      0,    306],
        [     3, 350816,      6,      0,      2,      2,    247,      0,      2],
        [   189,     35, 319873,    309,    136,    178,  19340,      2,    323],
        [  3776,      1,    463, 268922,   1229,      0,   8969,      0,    304],
        [   652,      5,    129,    821, 288120,      7,    561,      1,    313],
        [     0,    472,    176,      0,      0, 300310,    321,    555,     21],
        [   975,     30,  31306,  13358,    412,    231, 237046,      4,    304],
        [     0,    294,      0,      0,    214,      0,     36, 282561,      0],
        [     0,      5,      0,     20,     13,      0,    261,      0, 350775]])
Epoch end: Train, epoch number: 106
TrainEpoch/CM
 tensor([[266511,      1,    256,   3965,   1002,      0,    913,      0,    306],
        [     3, 350816,      6,      0,      2,      2,    247,   

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 108
ValidationEpoch/CM
 tensor([[269054,      1,    256,   3971,   1002,      0,    914,      0,    306],
        [     3, 354096,      6,      0,      2,      2,    247,      0,      2],
        [   189,     35, 323008,    311,    136,    178,  19383,      2,    323],
        [  3779,      1,    464, 271530,   1229,      0,   9007,      0,    304],
        [   652,      5,    129,    821, 290835,      7,    561,      1,    313],
        [     0,    472,    176,      0,      0, 303130,    321,    555,     21],
        [   977,     30,  31386,  13417,    412,    231, 239555,      4,    304],
        [     0,    294,      0,      0,    214,      0,     36, 285206,      0],
        [     0,      5,      0,     20,     13,      0,    261,      0, 354055]])


Monitored metric Validation/Accuracy did not improve in the last 10 records. Best score: 0.991. Signaling Trainer to stop.


Epoch end: Train, epoch number: 107
TrainEpoch/CM
 tensor([[269054,      1,    256,   3971,   1002,      0,    914,      0,    306],
        [     3, 354096,      6,      0,      2,      2,    247,      0,      2],
        [   189,     35, 323008,    311,    136,    178,  19383,      2,    323],
        [  3779,      1,    464, 271530,   1229,      0,   9007,      0,    304],
        [   652,      5,    129,    821, 290835,      7,    561,      1,    313],
        [     0,    472,    176,      0,      0, 303130,    321,    555,     21],
        [   977,     30,  31386,  13417,    412,    231, 239555,      4,    304],
        [     0,    294,      0,      0,    214,      0,     36, 285206,      0],
        [     0,      5,      0,     20,     13,      0,    261,      0, 354055]])


In [86]:

print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelMLP.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = subset.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/sklearn_linear/lightning_logs/version_4/checkpoints/epoch=97-step=4116.ckpt
tensor(0.9905, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[536,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0, 639,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 635,   0,   0,   0,   7,   0,   0],
        [  0,   0,   0, 503,   0,   0,  26,   0,   0],
        [  0,   0,   0,   0, 579,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 568,   0,   0,   0],
        [  0,   0,   9,   6,   0,   0, 522,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 514,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 610]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9905153512954712
      Test/F1Score          0.9901748895645142
        Test/Loss          0.025721527636051178
     Test/Precision         0

[{'Test/Loss': 0.025721527636051178,
  'Test/Accuracy': 0.9905153512954712,
  'Test/Precision': 0.9901461601257324,
  'Test/Recall': 0.9905153512954712,
  'Test/F1Score': 0.9901748895645142,
  'TestEpoch/Accuracy': 0.9902238249778748,
  'TestEpoch/Precision': 0.9905308485031128,
  'TestEpoch/Recall': 0.9902238249778748,
  'TestEpoch/F1Score': 0.9903081655502319}]

In [87]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/recursive')

x, y = next(iter(train_dataloader))
writer.add_graph(best_model, x)
writer.close()

In [94]:
cols = val.columns.to_numpy()

In [105]:
cols.tofile("rfe_selected_columns.npy", sep=",")